In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
import re
import itertools

import datetime
import nltk
import time

from shared_lib import utils, vocabulary
from shared_lib import ngram_lm
from shared_lib import ngram_utils
from shared_lib import simple_trigram
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from collections import OrderedDict

%matplotlib inline

In [3]:
boston_bomb_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/boston_comments_2013.03-2013.05.txt', lines=True)
boston_series_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/boston_comments_2013.10-2013.11.txt', lines=True)
colorado_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/colorado_comments_2017.06-2017.09.txt', lines=True)
florida_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/florida_comments_2017.06-2017.09.txt', lines=True)
houston_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/houston_comments_2017.06-2017.09.txt', lines=True)
miami_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/miami_comments_2017.06-2017.09.txt', lines=True)
nyc_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/nyc_comments_2012.08-2012.12.txt', lines=True)
puerto_rico_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/puerto_rico_comments_2017.06-2017.09.txt', lines=True)
vegas_df = pd.read_json('/Users/krista/Desktop/w266-project-master/data/reddit/vegas_comments_2017.06-2017.09.txt', lines=True)

In [4]:
# setup local times
boston_bomb_df['created_at_local'] = pd.to_datetime(boston_bomb_df['created_utc'], unit='s') \
                                     .dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
boston_series_df['created_at_local'] = pd.to_datetime(boston_series_df['created_utc'], unit='s') \
                                     .dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
colorado_df['created_at_local'] = pd.to_datetime(colorado_df['created_utc'], unit='s') \
                                     .dt.tz_localize('UTC').dt.tz_convert('US/Mountain')
florida_df['created_at_local'] = pd.to_datetime(florida_df['created_utc'], unit='s') \
                                     .dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
houston_df['created_at_local'] = pd.to_datetime(houston_df['created_utc'], unit='s') \
                                     .dt.tz_localize('UTC').dt.tz_convert('US/Central')
miami_df['created_at_local'] = pd.to_datetime(miami_df['created_utc'], unit='s') \
                                     .dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
nyc_df['created_at_local'] = pd.to_datetime(nyc_df['created_utc'], unit='s') \
                                 .dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
puerto_rico_df['created_at_local'] = pd.to_datetime(puerto_rico_df['created_utc'], unit='s') \
                                 .dt.tz_localize('UTC').dt.tz_convert('America/Puerto_Rico')
vegas_df['created_at_local'] = pd.to_datetime(vegas_df['created_utc'], unit='s') \
                                 .dt.tz_localize('UTC').dt.tz_convert('US/Pacific')  

In [5]:
#helper functions
def get_raw_text(df):
    raw_text = ''
    for post in df['body']:
        raw_text = ' '.join([raw_text, post])
    return raw_text

In [6]:
#boston_series_df.sort_values('created_at_local', ascending=True).head(5)

In [7]:
#sanity check for my stopword removal
# foo = ['foo']

# bar = ['bar', 'foo']

# man = [i for i in bar if i not in foo]

# print man

## Hurricane Irma (Florida/Miami), August 30th - Sept. 16

In [8]:
#tokenize words
tokenizer = RegexpTokenizer(r'\w+')

tokens = tokenizer.tokenize(get_raw_text(miami_df))

print tokens[:5]
token_feed = (utils.canonicalize_word(w) for w in tokens)

[u'gt', u'the', u'Tri', u'Rail', u'doesn']


In [9]:
# Collect counts of tokens and assign wordids.
vocab = vocabulary.Vocabulary(token_feed)
print "Vocabulary size: %d" % vocab.size

# Print out some (debugging) statistics to make sure everything went
# as we expected.  (Unsurprisingly, you should see "the" as the most popular word.)
print "Most common unigrams:"
for word, count in vocab.unigram_counts.most_common(20):
    print "\"%s\": %d" % (word, count)

Vocabulary size: 23761
Most common unigrams:
"the": 23230
"to": 16458
"i": 14764
"a": 14188
"and": 13497
"you": 10493
"it": 10006
"of": 9656
"in": 9652
"is": 8561
"that": 7640
"for": 6470
"s": 6322
"t": 5738
"on": 5071
"they": 4623
"are": 4577
"have": 4519
"be": 4315
"but": 4235


In [10]:
#remove single letters

tokens =[i for i in tokens if len(i)>1]

#lowercase everything

tokens = [ i.lower() for i in tokens]

In [11]:
from stop_words import get_stop_words

# create English stop words list
en_stop = get_stop_words('en')

# remove stop words from tokens from python stop_tokens
stopped_tokens = [i for i in tokens if i not in en_stop]

#remove stop words from nltk stop_tokens

stop_words = set(nltk.corpus.stopwords.words('english'))

#stopped_tokens = [i for i in tokens if i not in stop_words]

remove_phrases =["'s","n't", "'m","''", "''", "The", "https"]

#stopped_tokens = [ i for i in tokens if not in remove_phrases]

print stopped_tokens[:5]

texts= stopped_tokens

[u'gt', u'tri', u'rail', u'connect', u'two']


In [12]:
# Collect counts of tokens and assign wordids.
vocab = vocabulary.Vocabulary(stopped_tokens)
print "Vocabulary size: %d" % vocab.size

# Print out some (debugging) statistics to make sure everything went
# as we expected.  (Unsurprisingly, you should see "the" as the most popular word.)
print "Most common unigrams:"
for word, count in vocab.unigram_counts.most_common(20):
    print "\"%s\": %d" % (word, count)

Vocabulary size: 24465
Most common unigrams:
"miami": 2939
"people": 2588
"get": 2228
"like": 2222
"com": 1749
"power": 1732
"one": 1685
"would": 1588
"deleted": 1481
"good": 1362
"https": 1347
"go": 1343
"know": 1322
"amp": 1254
"still": 1194
"going": 1165
"back": 1146
"also": 1104
"time": 1097
"think": 1090


In [13]:
#some papers claimed stemming doesn't help
#from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
#p_stemmer = PorterStemmer()

# stem token
#texts = [p_stemmer.stem(i) for i in stopped_tokens]

# from pprint import pprint

# pprint(texts)


In [14]:
#convert to an array
texts_array = np.asarray(texts)
print type(texts_array)

print texts_array[:5]

<type 'numpy.ndarray'>
[u'gt' u'tri' u'rail' u'connect' u'two']


In [15]:
# Collect counts of tokens and assign wordids.
vocab = vocabulary.Vocabulary(texts)
print "Vocabulary size: %d" % vocab.size

# Print out some (debugging) statistics to make sure everything went
# as we expected.  (Unsurprisingly, you should see "the" as the most popular word.)
print "Most common unigrams:"
for word, count in vocab.unigram_counts.most_common(20):
    print "\"%s\": %d" % (word, count)

Vocabulary size: 24465
Most common unigrams:
"miami": 2939
"people": 2588
"get": 2228
"like": 2222
"com": 1749
"power": 1732
"one": 1685
"would": 1588
"deleted": 1481
"good": 1362
"https": 1347
"go": 1343
"know": 1322
"amp": 1254
"still": 1194
"going": 1165
"back": 1146
"also": 1104
"time": 1097
"think": 1090


In [16]:
#texts_array_irma = texts_arr

texts_irma=texts

texts_2= [u'hi']

texts_all=[texts_irma, texts_2]

dictionary = corpora.Dictionary(texts_all)

#print (texts[:5])
#doct_stream = (tokens for _, tokens in stopped_tokens)
#print texts_all[:5]

In [17]:
corpus = [dictionary.doc2bow(text) for text in texts_all]

In [18]:
ldamodel_100 = gensim.models.ldamodel.LdaModel(corpus, num_topics=100, id2word= dictionary, passes= 20)
print(ldamodel_100.print_topics(num_topics=10, num_words=10))

[(42, u'0.000*"people" + 0.000*"miami" + 0.000*"like" + 0.000*"get" + 0.000*"com" + 0.000*"deleted" + 0.000*"would" + 0.000*"power" + 0.000*"one" + 0.000*"good"'), (29, u'0.000*"miami" + 0.000*"get" + 0.000*"people" + 0.000*"one" + 0.000*"com" + 0.000*"deleted" + 0.000*"like" + 0.000*"https" + 0.000*"think" + 0.000*"would"'), (31, u'0.000*"miami" + 0.000*"like" + 0.000*"people" + 0.000*"get" + 0.000*"one" + 0.000*"deleted" + 0.000*"com" + 0.000*"power" + 0.000*"would" + 0.000*"https"'), (73, u'0.007*"souls" + 0.004*"boos" + 0.004*"bc0k8qyr" + 0.004*"413f" + 0.004*"coffeeshops" + 0.004*"antarctica" + 0.004*"0565732" + 0.004*"floodlights" + 0.004*"debunked" + 0.004*"paragraphs"'), (32, u'0.009*"miami" + 0.008*"people" + 0.007*"like" + 0.006*"get" + 0.005*"com" + 0.005*"one" + 0.005*"would" + 0.005*"power" + 0.004*"https" + 0.004*"go"'), (96, u'0.000*"miami" + 0.000*"like" + 0.000*"people" + 0.000*"com" + 0.000*"deleted" + 0.000*"go" + 0.000*"would" + 0.000*"one" + 0.000*"power" + 0.000*"

In [19]:
ldamodel_50 = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word= dictionary)
print(ldamodel_50.print_topics(num_topics=10, num_words=10))

[(15, u'0.007*"miami" + 0.007*"people" + 0.007*"like" + 0.007*"get" + 0.006*"power" + 0.005*"com" + 0.005*"one" + 0.004*"know" + 0.004*"go" + 0.004*"still"'), (14, u'0.008*"people" + 0.008*"get" + 0.007*"miami" + 0.005*"like" + 0.005*"power" + 0.005*"amp" + 0.004*"think" + 0.004*"one" + 0.004*"com" + 0.004*"would"'), (25, u'0.008*"miami" + 0.007*"people" + 0.006*"like" + 0.006*"one" + 0.006*"com" + 0.006*"get" + 0.005*"deleted" + 0.004*"power" + 0.004*"know" + 0.004*"good"'), (16, u'0.008*"miami" + 0.008*"people" + 0.005*"power" + 0.005*"one" + 0.005*"get" + 0.004*"would" + 0.004*"https" + 0.004*"know" + 0.004*"also" + 0.004*"still"'), (37, u'0.009*"miami" + 0.008*"people" + 0.006*"power" + 0.006*"get" + 0.006*"https" + 0.005*"deleted" + 0.004*"go" + 0.004*"going" + 0.004*"like" + 0.004*"one"'), (32, u'0.010*"miami" + 0.008*"people" + 0.006*"com" + 0.006*"would" + 0.006*"like" + 0.005*"one" + 0.005*"get" + 0.004*"amp" + 0.004*"deleted" + 0.004*"good"'), (38, u'0.007*"get" + 0.007*"miam

In [20]:
# doc_stream = (tokens for _, tokens in token_feed)  # generator
# test_docs = list(itertools.islice(doc_stream, 8000, 9000))

In [21]:
# def intra_inter(model, test_docs, num_pairs=10000):
#     # split each test document into two halves and compute topics for each half
#     part1 = [model[dictionary.doc2bow(tokens[: len(tokens) / 2])] for tokens in test_docs]
#     part2 = [model[dictionary.doc2bow(tokens[len(tokens) / 2 :])] for tokens in test_docs]
    
#     # print computed similarities (uses cossim)
#     print("average cosine similarity between corresponding parts (higher is better):")
#     print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

#     random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
#     print("average cosine similarity between 10,000 random parts (lower is better):")    
#     print(np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [22]:
# print("LDA results:")
# intra_inter(ldamodel_25, test_docs)

In [23]:
ldamodel_25 = gensim.models.ldamodel.LdaModel(corpus, num_topics=25, id2word= dictionary)
print(ldamodel_25.print_topics(num_topics=10, num_words=10))

[(17, u'0.010*"miami" + 0.008*"like" + 0.007*"people" + 0.006*"get" + 0.006*"com" + 0.005*"power" + 0.005*"one" + 0.004*"would" + 0.004*"good" + 0.004*"back"'), (15, u'0.008*"get" + 0.007*"like" + 0.006*"people" + 0.005*"power" + 0.004*"one" + 0.004*"miami" + 0.004*"com" + 0.004*"back" + 0.004*"deleted" + 0.004*"also"'), (14, u'0.009*"like" + 0.008*"miami" + 0.007*"people" + 0.005*"get" + 0.005*"one" + 0.005*"com" + 0.004*"know" + 0.004*"time" + 0.004*"power" + 0.004*"deleted"'), (11, u'0.008*"get" + 0.007*"miami" + 0.007*"people" + 0.007*"like" + 0.005*"would" + 0.004*"think" + 0.004*"power" + 0.004*"one" + 0.004*"com" + 0.004*"amp"'), (19, u'0.009*"people" + 0.007*"miami" + 0.007*"com" + 0.006*"get" + 0.006*"power" + 0.005*"would" + 0.004*"good" + 0.004*"like" + 0.004*"https" + 0.004*"amp"'), (16, u'0.009*"miami" + 0.007*"people" + 0.006*"like" + 0.005*"get" + 0.005*"com" + 0.005*"one" + 0.004*"go" + 0.004*"would" + 0.004*"deleted" + 0.004*"good"'), (0, u'0.007*"miami" + 0.006*"peopl

In [24]:
hlda = models.HdpModel(corpus,id2word= dictionary )
print(hlda.print_topics(num_topics=10, num_words=10))

[(0, u'0.009*miami + 0.008*people + 0.007*get + 0.007*like + 0.005*com + 0.005*power + 0.005*one + 0.005*would + 0.004*deleted + 0.004*good'), (1, u'0.000*ghost + 0.000*duration + 0.000*1650 + 0.000*unless + 0.000*qualified + 0.000*rest + 0.000*valuable + 0.000*formula + 0.000*lbs + 0.000*albums'), (2, u'0.000*flamingo + 0.000*rituals + 0.000*ripped + 0.000*wynn + 0.000*shoulders + 0.000*playwright + 0.000*hollywood + 0.000*11e6a95042844e2f9ad5a111c5ffd214 + 0.000*plancha + 0.000*coach'), (3, u'0.000*alreayd + 0.000*randomly + 0.000*it_could_happen_here + 0.000*contrary + 0.000*buckling + 0.000*x5q0eoxa_zi + 0.000*effing + 0.000*awoken + 0.000*dispatcher + 0.000*twelve'), (4, u'0.000*americana + 0.000*shouted + 0.000*hurricaneshutter5 + 0.000*goose + 0.000*vlad + 0.000*stimulus + 0.000*device + 0.000*tears + 0.000*mangos + 0.000*20report'), (5, u'0.000*dedicated + 0.000*dilapidated + 0.000*sb + 0.000*citizenship + 0.000*toilet + 0.000*autistic + 0.000*adulting + 0.000*thornton + 0.000*

In [25]:
hdp = models.HdpModel(corpus, id2word=dictionary)
print(hdp.print_topics(num_topics=10, num_words=10))

[(0, u'0.009*miami + 0.008*people + 0.007*get + 0.007*like + 0.005*com + 0.005*power + 0.005*one + 0.005*would + 0.004*deleted + 0.004*good'), (1, u'0.000*says + 0.000*shades + 0.000*tooooo + 0.000*hurdle + 0.000*240 + 0.000*jean + 0.000*tvs + 0.000*aatzpil + 0.000*358 + 0.000*disorders'), (2, u'0.003*hi + 0.000*cream + 0.000*detour + 0.000*paraiso + 0.000*chef + 0.000*oftentimes + 0.000*dump + 0.000*camping + 0.000*homophobic + 0.000*advises'), (3, u'0.000*inelastic + 0.000*exxon + 0.000*bye + 0.000*wrenching + 0.000*jalousie + 0.000*melts + 0.000*go + 0.000*11e6a95042844e2f9ad5a111c5ffd214 + 0.000*louder + 0.000*issuing'), (4, u'0.000*shoulders + 0.000*laying + 0.000*vida + 0.000*crowlers + 0.000*sewn + 0.000*insanity + 0.000*ripping + 0.000*windshield + 0.000*diminishing + 0.000*holds'), (5, u'0.000*divisive + 0.000*cramps + 0.000*emily + 0.000*6z0w20 + 0.000*gorgeous + 0.000*plead + 0.000*breather + 0.000*frost + 0.000*webm + 0.000*courteously'), (6, u'0.000*alabam + 0.000*usps + 0

 ## Hurricane Harvey (Houston), Around August 25

In [26]:
#houston_subset_df = houston_df[houston_df['created_at_local'].dt.date > datetime.date(2017, 7, 15)]
df_size = houston_df.shape[0]
chosen_idx = np.random.choice(df_size, replace=False, size=df_size/4)
houston_subset_df = houston_df.iloc[chosen_idx]
houston_subset_df.shape

(64354, 26)

In [27]:


#tokenize words
tokenizer = RegexpTokenizer(r'\w+')

#do some preprocessing before tokenizing
#clean_data=get_raw_text(houston_subset_df).lower()

#create tokens
tokens = nltk.word_tokenize(get_raw_text(houston_subset_df))

token_feed = (utils.canonicalize_word(w) for w in tokens)

# Collect counts of tokens and assign wordids.
vocab = vocabulary.Vocabulary(token_feed)
print "Vocabulary size: %d" % vocab.size

# Print out some (debugging) statistics to make sure everything went
# as we expected.  (Unsurprisingly, you should see "the" as the most popular word.)
print "Most common unigrams:"
for word, count in vocab.unigram_counts.most_common(10):
    print "\"%s\": %d" % (word, count)

Vocabulary size: 52958
Most common unigrams:
".": 102008
"the": 63488
",": 50601
"to": 44546
"i": 40524
"a": 37558
"and": 34987
"it": 27325
"you": 27041
"of": 26645


In [28]:
#Preprocessing

#remove single letters

tokens =[i for i in tokens if len(i)>1]


tokens = [ i.lower() for i in tokens]




#remove stop words

# create English stop words list
en_stop = get_stop_words('en')

# remove stop words from tokens from python stop_tokens
stopped_tokens = [i for i in tokens if i not in en_stop]

#remove stop words from nltk stop_tokens

stop_words = set(nltk.corpus.stopwords.words('english'))

stopped_tokens = [i for i in tokens if i not in stop_words]


#word for word in word_list if word not in stopwords.words('english')]

print stopped_tokens[:5]

texts= stopped_tokens

[u'yes', u'maybe', u'phrasing', u"n't", u'right']


In [29]:
# Collect counts of tokens and assign wordids.
vocab = vocabulary.Vocabulary(stopped_tokens)
print "Vocabulary size: %d" % vocab.size

# Print out some (debugging) statistics to make sure everything went
# as we expected.  (Unsurprisingly, you should see "the" as the most popular word.)
print "Most common unigrams:"
for word, count in vocab.unigram_counts.most_common(20):
    print "\"%s\": %d" % (word, count)

Vocabulary size: 55099
Most common unigrams:
"'s": 16441
"n't": 15241
"people": 7203
"like": 6631
"get": 6381
"'m": 5548
"would": 5047
"'re": 4561
"one": 4440
"''": 4436
"houston": 4282
"deleted": 4139
"``": 4096
"...": 4044
"https": 3758
"know": 3698
"good": 3526
"go": 3373
"water": 3227
"going": 3101


In [30]:
#convert to an array
texts_array = np.asarray(texts)

print texts_array[:5]

texts_harvey=texts

texts_2= [u'hi']

texts_all=[texts_harvey, texts_2]

dictionary = corpora.Dictionary(texts_all)


#convert to corupus
corpus = [dictionary.doc2bow(text) for text in texts_all]

[u'yes' u'maybe' u'phrasing' u"n't" u'right']


In [31]:
ldamodel_50 = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word= dictionary)
print(ldamodel_50.print_topics(num_topics=10, num_words=10))

[(39, u'0.022*"\'s" + 0.015*"n\'t" + 0.008*"people" + 0.008*"like" + 0.007*"would" + 0.007*"\'m" + 0.006*"get" + 0.005*"..." + 0.004*"\'\'" + 0.004*"``"'), (18, u'0.021*"\'s" + 0.017*"n\'t" + 0.008*"like" + 0.006*"\'m" + 0.006*"people" + 0.006*"houston" + 0.006*"get" + 0.006*"deleted" + 0.005*"would" + 0.005*"..."'), (1, u'0.015*"n\'t" + 0.015*"\'s" + 0.007*"like" + 0.007*"people" + 0.006*"get" + 0.005*"deleted" + 0.005*"\'m" + 0.005*"would" + 0.004*"``" + 0.004*"\'\'"'), (0, u'0.015*"\'s" + 0.014*"n\'t" + 0.007*"like" + 0.007*"people" + 0.006*"get" + 0.005*"\'\'" + 0.005*"would" + 0.005*"one" + 0.005*"``" + 0.005*"..."'), (20, u'0.018*"\'s" + 0.017*"n\'t" + 0.008*"people" + 0.008*"get" + 0.007*"like" + 0.005*"\'re" + 0.005*"think" + 0.005*"\'\'" + 0.005*"https" + 0.005*"deleted"'), (13, u'0.018*"n\'t" + 0.011*"\'s" + 0.007*"people" + 0.006*"like" + 0.005*"..." + 0.005*"\'m" + 0.005*"\'\'" + 0.005*"would" + 0.005*"know" + 0.005*"get"'), (33, u'0.015*"\'s" + 0.013*"n\'t" + 0.010*"people

In [32]:
hlda = models.HdpModel(corpus,id2word= dictionary )
print(hlda.print_topics(num_topics=10, num_words=10))

[(0, u"0.018*'s + 0.017*n't + 0.008*people + 0.007*like + 0.007*get + 0.006*'m + 0.006*would + 0.005*'re + 0.005*one + 0.005*''"), (1, u'0.002*hi + 0.000*relaxing + 0.000*calder + 0.000*105k + 0.000*5000 + 0.000*out-right + 0.000*straps + 0.000*unusually + 0.000*fairways + 0.000*//youtu.be/ec5mrgjjan8'), (2, u'0.000*sublease + 0.000*diddly-poo + 0.000*//www.reddit.com/r/houston/comments/6yngn3/houstons_undocumented_immigrants_left_destitute/dmp61n5/ + 0.000*oh.. + 0.000*heather + 0.000*1,000s + 0.000*urging + 0.000***hank + 0.000*licencia + 0.000*equis'), (3, u'0.000*mens + 0.000*mavic + 0.000*00:00:00 + 0.000*eu + 0.000*.5 + 0.000*windmill + 0.000*pro-tip + 0.000*impersonating + 0.000*rpx + 0.000*overflowing'), (4, u"0.000*2230 + 0.000*releases* + 0.000*confidence + 0.000*srew + 0.000*unfiltered + 0.000*afterlife + 0.000*wsj + 0.000*ditch/i'm-not-sure-technically-what-to-call-it + 0.000*//imgur.com/z6azbhl + 0.000*//goo.gl/maps/4ywt92lp3fk"), (5, u'0.000*paranoid1123 + 0.000*281-948-2

In [33]:
hlda = models.HdpModel(corpus,id2word= dictionary )
print(hlda.print_topics(num_topics=10, num_words=10))

[(0, u"0.018*'s + 0.017*n't + 0.008*people + 0.007*like + 0.007*get + 0.006*'m + 0.006*would + 0.005*'re + 0.005*one + 0.005*''"), (1, u"0.000*probing + 0.000*showtime + 0.000*f**d + 0.000*o'rourke + 0.000*invision + 0.000*\U0001f602\U0001f602\U0001f602 + 0.000*probation + 0.000*detained + 0.000*//www.nhc.noaa.gov + 0.000*stuffing"), (2, u'0.000*showbiz + 0.000*steroids + 0.000*ll=29.896615075309754 + 0.000*copper + 0.000*poll + 0.000*med + 0.000*messaged + 0.000*euless + 0.000*www.connollyshireman.com + 0.000*pappasito'), (3, u'0.000*salons + 0.000*pro-net + 0.000*brah + 0.000*top** + 0.000*hitchhiking + 0.000*2097/07184 + 0.000*to.. + 0.000*8/25 + 0.000*dingaling + 0.000*to=remindmebotwrangler'), (4, u'0.000*somalian + 0.000*ejs + 0.000*understandings + 0.000*//en.wikipedia.org/wiki/levee_breach + 0.000*jac + 0.000*jesse + 0.000*afaik + 0.000*abdul + 0.000*tcr + 0.000*sadistic'), (5, u'0.000*13,300 + 0.000*frequenter + 0.000*acoustic + 0.000*edgy + 0.000*thoughtful + 0.000*info\u2026

### Las Vegas Shooting, October 1, 2017

In [ ]:
#References
https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_coherence_model_selection.ipynb
    